In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from scipy.stats import skew
from math import sqrt
from sklearn import linear_model
import statsmodels.formula.api as smf

warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_columns', 300)

%matplotlib inline
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 300)

In [16]:
df_test = pd.read_csv('C:\\Users\\STEVEN\\Documents\\Thinkful\\Files\\Housing\\housing_test.csv', index_col='Id')
df_train = pd.read_csv('C:\\Users\\STEVEN\\Documents\\Thinkful\\Files\\Housing\\train.csv', index_col='Id')

In [17]:
y_train = df_train.SalePrice
df_train = df_train.drop(columns='SalePrice')

In [18]:
from sklearn.preprocessing import Imputer

# Replace NaN with corresponding Neighborhood Median
df_train["LotFrontage"] = df_train.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))
df_test["LotFrontage"] = df_test.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))
    
# Replace NaN with Feature Median 
numerical_features = df_train.select_dtypes(include='number').columns
my_imputer = Imputer()
df_train[numerical_features] = my_imputer.fit_transform(df_train[numerical_features])
df_test[numerical_features] = my_imputer.transform(df_test[numerical_features])

In [19]:
idx_split = df_train.shape[0]
df_train = df_train.append(df_test)

In [20]:
NumStr = ["MSSubClass","BsmtFullBath","BsmtHalfBath","HalfBath","BedroomAbvGr","KitchenAbvGr","MoSold","YrSold","YearBuilt","YearRemodAdd","LowQualFinSF","GarageYrBlt"]
for col in NumStr:
    df_train[col]=df_train[col].astype(str)
    
non_ordinal_features = ['LandSlope','Utilities','MSZoning', 'Street','Alley','LotShape','LandContour','LotConfig','Neighborhood','Condition1','Condition2','BldgType','HouseStyle', 'RoofStyle','RoofMatl','Exterior1st', 'Exterior2nd','MasVnrType','Foundation', 'BsmtExposure','BsmtFinType1','BsmtFinType2', 'Heating', 'CentralAir', 'Electrical','Functional', 'GarageType','GarageFinish','PavedDrive', 'Fence','MiscFeature','SaleType','SaleCondition', "MSSubClass","BsmtFullBath","BsmtHalfBath","HalfBath","BedroomAbvGr","KitchenAbvGr","MoSold","YrSold","YearBuilt","YearRemodAdd","LowQualFinSF","GarageYrBlt"]
ordinal_features = df_train.select_dtypes(include='object').columns.drop(non_ordinal_features)


In [21]:
print(ordinal_features)

Index(['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC',
       'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC'],
      dtype='object')


In [22]:
dummies = pd.get_dummies(df_train.loc[:,non_ordinal_features], drop_first=True)
df_train = pd.concat([df_train,dummies], axis=1)
df_train = df_train.drop(non_ordinal_features,axis=1)

In [23]:
def cat_to_num(x):
    if x=='Ex':
        return 5
    if x=='Gd':
        return 4
    if x=='TA':
        return 3
    if x=='Fa':
        return 2
    if x=='Po':
        return 1
    if x=='None':
        return 0

df_train.loc[:,ordinal_features] = df_train.loc[:,ordinal_features].applymap(cat_to_num)

In [24]:
X = df_train.loc[:idx_split, :]
X_test = df_train.loc[idx_split+1:, :]

In [25]:
X = X.assign(SalePrice=y_train)

In [26]:
X["TotalHouse"] = X["TotalBsmtSF"] + X["1stFlrSF"] + X["2ndFlrSF"]   
X["TotalHouse_OverallQual"] = X["TotalHouse"] * X["OverallQual"]
X["GrLivArea_OverallQual"] = X["GrLivArea"] * X["OverallQual"]


X_test["TotalHouse"] = X_test["TotalBsmtSF"] + X_test["1stFlrSF"] + X_test["2ndFlrSF"]   
X_test["TotalHouse_OverallQual"] = X_test["TotalHouse"] * X_test["OverallQual"]
X_test["GrLivArea_OverallQual"] = X_test["GrLivArea"] * X_test["OverallQual"]


In [27]:
D = X['ExterQual'].values.reshape(-1,1)
Z = X.drop(columns='ExterQual')
Z2 = Z.dropna(axis=1)
D2 = D
Z3 = Z2

In [35]:
from sklearn import tree

from IPython.display import Image
from sklearn.model_selection import cross_val_score
#import pydotplus
import time

start_time = time.time()

dct = tree.DecisionTreeClassifier(
    max_features=1,
    max_depth=4,
)
dct.fit(Z2, D)

print(cross_val_score(dct, Z2, D, cv=10))



print("--- %s seconds ---" % (time.time() - start_time))

[0.61486486 0.61486486 0.61904762 0.62585034 0.7260274  0.63013699
 0.62068966 0.62068966 0.625      0.73611111]
--- 0.21442842483520508 seconds ---


In [36]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
import time
start_time = time.time()
rfc = ensemble.RandomForestClassifier()
rfc.fit(Z3, D2)

print(cross_val_score(rfc, Z3, D2, cv=10))


print("--- %s seconds ---" % (time.time() - start_time))

[0.82432432 0.87837838 0.87755102 0.89795918 0.87671233 0.84246575
 0.91034483 0.86896552 0.89583333 0.875     ]
--- 0.8257913589477539 seconds ---
